In [1]:
import os
import urllib
import csv
import glob
import numpy as np
import pandas as pd
from pandas import concat
from pandas import DataFrame
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn import datasets, linear_model
import scipy.io as sio
import datetime
import requests
from dateutil.relativedelta import relativedelta
import contextily as ctx
from matplotlib.colors import LinearSegmentedColormap
from datetime import datetime, timezone
from mintpy.utils import readfile, writefile
from mintpy.defaults.plot import *
from mintpy.objects.gps import search_gps, GPS
from mintpy.objects import sensor
from mintpy.utils import utils as ut
from mintpy.view import prep_slice, plot_slice
from mintpy.cli import view, timeseries2velocity
%load_ext jupyter_ai
# import utils.gps
# import utils.insar
# import utils.plot_functions
from utils.plot_functions import get_basemap
from utils.plot_functions import modify_colormap
from utils.seismicity import get_earthquakes
from utils.seismicity import normalize_earthquake_times
from utils.gps import get_GPS


ModuleNotFoundError: No module named 'plot_data'

In [ ]:
# Hard-wired and optional parameters
data_dir = os.getenv('SCRATCHDIR') + '/MaunaLoa/MLtry/data/'
dem_file = data_dir + 'demGeo.h5'
lines=sio.loadmat(data_dir + '/hawaii_lines_new.mat',squeeze_me=True);
GPS_list_file = data_dir + 'GPS_BenBrooks_03-05full.txt'
work_dir = os.path.expanduser('~/offline/MaunaLoaSenDT87/mintpy/geo')
depth_range="0 10"
cmap_name = "plasma_r"; exclude_beginning = 0.2; exclude_end = 0.2
plot_box =[19.29, 19.6, -155.79, -155.41]

date12 = '20221216_20230202'
start_date, end_date, scale_fac, key_length, unit ="20180905", "20201229", 500, 4, "cm"


In [ ]:
# get dem, earthquake and GPS data, normalize event times for plotting)
dem_shade, dem_extent = get_basemap(dem_file)
events_df = get_earthquakes(start_date, end_date, plot_box)
# norm_times = normalize_earthquake_times(events_df, start_date, end_date)
norm_times = normalize_earthquake_times(events_df, start_date, end_date)
cmap = modify_colormap( cmap_name = cmap_name, exclude_beginning = exclude_beginning, exclude_end = exclude_end, show = False)

gps,lon,lat,U,V,Z,quiver_label = get_GPS(GPS_list_file, plot_box, start_date, end_date, unit, key_length)

data_dir1 = "MaunaLoaSenAT124/mintpy/S1_IW23_124_0059_0063_20180905_XXXXXXXX_N19122_N19785_W155940_W155223.he5"
data_dir1 = os.getenv('SCRATCHDIR') + '/MaunaLoaSenAT124/mintpy/S1_IW23_124_0059_0063_20180905_XXXXXXXX_N19122_N19785_W155940_W155223.he5'
data_dir1 = "MaunaLoaSenAT124"

plot_type = 'ifgram' 
plot_type = 'shaded_relief' 
plot_type = 'timeseries' 

In [ ]:
%%capture
if plot_type is 'timeseries':
    eos_file, out_dir, vel_file = get_eos_file(data_dir1)
    cmd = f'{eos_file} --start-date {start_date} --end-date {end_date} --output {vel_file}'
    timeseries2velocity.main( cmd.split() )

In [ ]:
# Cell to plot GPS on shaded relief or interferogram
fig, ax = plt.subplots(figsize=[12, 5] )

if plot_type is 'shaded_relief':
    plot_shaded_relief(ax, dem_file, plot_box = plot_box)
else:
    if plot_type is 'timeseries':
        cmd = generate_view_velocity_cmd(vel_file, plot_box)
    elif plot_type is 'ifgram':
        cmd = generate_view_ifgram_cmd(work_dir, date12, plot_box)
    data, atr, inps = prep_slice(cmd)
    ax, inps, im, cbar = plot_slice(ax, data, atr, inps)

# plot lines and events
ax.scatter(events_df["Longitude"],events_df["Latitude"],s=2*events_df["Magnitude"] ** 3,
         c=norm_times,cmap=cmap,alpha=0.8)
add_colorbar(cmap = cmap, start_date = start_date, end_date = end_date)
ax.set_title('Events:  ' + start_date + ' - ' + end_date);
quiv=ax.quiver(lon, lat, U, V, scale = scale_fac, color='blue')
ax.quiverkey(quiv, -155.50, 19.57, key_length*10 , quiver_label, labelpos='N',coordinates='data',
                  color='blue',fontproperties={'size': 20}) 

In [ ]:
# # print(eos_file)
# atr = readfile.read_attribute(eos_file)
# mask, dict_mask = readfile.read(eos_file, datasetName="/HDFEOS/GRIDS/timeseries/quality/mask")
# # temporal_coherence, dict_temp_coh = readfile.read(eos_file, datasetName="/HDFEOS/GRIDS/timeseries/quality/temporalCoherence")

# atr['FILE_TYPE'] = 'mask'
# mask_file = vel_file.replace('velocity','maskTempCoh')
# writefile.write(mask, out_file=mask_file, metadata=atr)

# fig, ax = plt.subplots(figsize=[12, 5])
# cmd = generate_view_velocity_cmd(vel_file, mask_file, plot_box)
# #cmd = generate_view_velocity2_cmd(vel_file, mask_file, plot_box)

# print(cmd)
# data, atr, inps = prep_slice(cmd)
# ax, inps, im, cbar = plot_slice(ax, data, atr, inps)